#### Script para adequar a base de Terra Indígena

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [1]:
# Módulos necessários
import geopandas as gpd
import pandas as pd
import os
import glob
import numpy as np

In [2]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir diretório principal
# # dirpath = input('Diretório principal: ')
# dirpath = os.getenv('INPUT_PATH', '/app/input')
# 
dirpath = os.getenv('INPUT_PATH', '/app/input')


In [3]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir caminho do shapefile original de Terra Indígena (tis_poligonais.shp)
# # ti = input('Caminho Terra Indígena: ')
# ti_path = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'Terras_Indigenas_Municipios')
# 
ti_path = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'Terras_Indigenas_Municipios')


In [4]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir caminho do shapefile de limite do Brasil
# # br = input('Caminho do limite do Brasil: ')
# br = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'br')
# 
br = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'br')


In [5]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - sem Buffer)
# # limites = input('Caminho Municípios: ')
# mun_path = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'Municipios', 'BR_Municipios_2021.shp')
# 
mun_path = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'Municipios', 'BR_Municipios_2021.shp')


In [7]:
# Ler o arquivo original de Terra Indígena (tis_poligonais.shp)base = gpd.read_file(os.path.join(ti_path, os.getenv('TI_FILE', 'tis_poligonais.shp')))base = base.to_crs(epsg=4326)# Ler limite do Brasilbrasil = gpd.read_file(br)brasil = brasil.to_crs(epsg=4326)# Lendo shapefile de Municípioslim = gpd.read_file(mun_path)lim = lim.to_crs(epsg=4326)# Criar pasta para armazenar a base de Terra Indígena recortada por estadouf_path = os.path.join(dirpath, os.getenv('TI_ESTADOS_OUTPUT', 'Terras_Indigenas_Estados'))os.makedirs(uf_path, exist_ok=True)# Criar pasta para armazenar a base de Terra Indígena recortada por municípiosti_mun_path = os.path.join(dirpath, os.getenv('TI_MUNICIPIOS_OUTPUT', 'Terras_Indigenas_Municipios'))os.makedirs(ti_mun_path, exist_ok=True)os.makedirs(mun_path, exist_ok=True)# Adicionar coluna TERRIND_Cbase['CD_TERRIND'] = '1'# Listar as colunas do dataframecolunas = list(base.columns)colunas = [coluna for coluna in colunas if 'CD_TERRIND' not in coluna and 'terrai_cod' not in coluna and 'geometry' not in coluna]# Excluindo as colunas desnecessáriasbase = base.drop(colunas, axis = 1)# Fazer a diferença entre a base de terras indígenas e o limite do Brasilerase = brasil.overlay(base, how='difference', keep_geom_type=True)# Transformar para Single Partsingle = erase.explode(ignore_index=True)# Merge da terra indígena com o erasemerge = pd.concat([base, single]).pipe(gpd.GeoDataFrame)# Excluindo colunas desnecessáriasmerge = merge.drop(['NM_PAIS', 'AREA_KM2'], axis = 1)# Preencher os valores nulos por 0merge.update(merge['CD_TERRIND'].fillna('0'))# Cortar por estado# Agrupar por estadoagrupado = lim.groupby('SIGLA_UF')for key,values in agrupado:    output = uf_path + f'\TI_2022_{key}.shp'    mun = lim[lim["SIGLA_UF"] == f"{key}"]    geodf_clip = gpd.clip(merge, mun)    geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))# Lista dos estadosestados = list(np.unique(lim['SIGLA_UF']))# Cortar por municípiofor estado in estados:    shapes = glob.glob(uf_path + f'**/*{estado}.shp')    # Selecionar GeoDataFrame por estado    select = lim[lim['SIGLA_UF'] == f'{estado}']    # Agrupar por município    agrupado = select.groupby('CD_MUN')    for shape in shapes:        for key,values in agrupado:            ti = gpd.read_file(shape)            ti = ti.to_crs(epsg=4326)            nome_arq = os.path.basename(shape)            nome_arq = nome_arq.replace('.shp', f'_{key}.shp')output = ti_mun_path + f"\{nome_arq}"            mun = lim[lim["CD_MUN"] == f"{key}"]            geodf_clip = gpd.clip(ti, mun)            geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))